<h3 style="text-align: center;"><span style="color:red">This example that can be overwritten by container updates.</span></h3>
<h4 style="text-align: center;"><span style="color:red">Please create a new notebook if you plan to make changes.</span></h4>

## Intro

This example jupyter file demonstrates basic [stable diffusion](https://huggingface.co/docs/diffusers/api/pipelines/stable_diffusion/stable_diffusion_2#texttoimage) functionality.

## Getting started

Run each cell to step through the process, including (first-time only) downloading a model to use.
Note: Run cells, in order, at least once, to set up the environment.  Once set up, you can change the prompt and rerun just that cell.  If you make changes in a cell, that cell should be rerun for the changes to take effect.

In [ ]:
from dataclasses import dataclass, field, asdict
import torch

@dataclass
class Model:
    name: str
    repo_id: str
    variant: str = None
    torch_dtype: str = None
    foldername: str = field(init=False)

    def __post_init__(self):
        # Set the value of foldername based on repo_id
        self.foldername = self.repo_id.split("/")[1]

    def fp16(self):
        self.variant = "fp16"
        self.torch_dtype = torch.float16
        return self

    def pretrain_options(self):
        return self._filter(["name", "repo_id", "foldername"])

    def _filter(self, excludes):
        # Return the members that aren't set to None or in excludes
        return {k:v for k,v in asdict(self).items() if (v is not None and k not in excludes)}

models = {
    "sd2"        : Model("sd2", "stabilityai/stable-diffusion-2-base"),
    "sd-turbo"   : Model("sd-turbo", "stabilityai/sd-turbo"),
    "sdxl"       : Model("sdxl", "stabilityai/stable-diffusion-xl-base-1.0"),
}

model = models["sd2"].fp16()
model

In [ ]:
from diffusers import AutoPipelineForText2Image

pipe = AutoPipelineForText2Image.from_pretrained(model.repo_id, **model.pretrain_options())
pipe.to("cuda")

prompt = "A cinematic shot of a baby racoon wearing an intricate italian priest robe."
image = pipe(prompt=prompt).images[0]
# image = pipe(prompt=prompt, num_inference_steps=25, guidance_scale=0.0).images[0]
image

In [ ]:
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
import torch

repo_id = "stabilityai/stable-diffusion-2-base"
pipe = DiffusionPipeline.from_pretrained(repo_id, torch_dtype=torch.float16, variant="fp16")

pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe = pipe.to("cuda")

prompt = "High quality photo of an astronaut riding a horse in space"
image = pipe(prompt, num_inference_steps=25).images[0]
image

In [ ]:
from diffusers import StableDiffusionPipeline
import torch

# Check out https://huggingface.co/sd-dreambooth-library for loads of models from the community
model_id = "sd-dreambooth-library/mr-potato-head"

# Load the pipeline
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16).to("cuda")

In [ ]:
# prompt = "an abstract oil painting of sks mr potato head by picasso"
prompt = "anime carrie fisher hologram with R2-D2"
image = pipe(prompt, num_inference_steps=50, guidance_scale=7.5).images[0]
image

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sd-turbo", torch_dtype=torch.float16, variant="fp16")
pipe.to("cuda")

# prompt = "A cinematic shot of a baby racoon wearing an intricate italian priest robe."
prompt = "anime carrie fisher hologram with R2-D2"
image = pipe(prompt=prompt, num_inference_steps=1, guidance_scale=0.0).images[0]
image

In [ ]:
from huggingface_hub import hf_hub_download
from huggingface_hub import snapshot_download

snapshot_download(repo_id="stabilityai/sd-turbo", local_dir="/app/data/stabilityai/sd-turbo")

In [ ]:
from diffusers import AutoPipelineForText2Image
import torch

# pipe = AutoPipelineForText2Image.from_pretrained("/app/data/stabilityai/sd-turbo", torch_dtype=torch.float16, variant="fp16")
pipe = AutoPipelineForText2Image.from_pretrained("/app/data/stabilityai/sd-turbo")
pipe.to("cuda")

# prompt = "A cinematic shot of a baby racoon wearing an intricate italian priest robe."
prompt = "A cute, fuzzy kitten fighting off a hawk with a slingshot."
# prompt = "anime carrie fisher hologram with R2-D2"
image = pipe(prompt=prompt, num_inference_steps=2, guidance_scale=0.0).images[0]
image

In [ ]:
import torch
from diffusers import DiffusionPipeline, AutoencoderKL

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", vae=vae, torch_dtype=torch.float16, variant="fp16", use_safetensors=True)
pipe.to("cuda")

# refiner = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-refiner-1.0", vae=vae, torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
# refiner.to("cuda")

n_steps = 40
high_noise_frac = 0.7

prompt = "A majestic lion jumping from a big stone at night"

pipe.enable_attention_slicing()
n_steps = 10
image = pipe(prompt=prompt, num_inference_steps=n_steps, denoising_end=high_noise_frac, output_type="latent", height=128, width=128).images
# image = pipe(prompt=prompt, num_inference_steps=n_steps, denoising_end=high_noise_frac, output_type="latent").images
# image = refiner(prompt=prompt, num_inference_steps=n_steps, denoising_start=high_noise_frac, image=image).images[0]
image